In [8]:
from datasets import load_dataset

datasets_path = "./datasets/"

datasets = load_dataset(path=datasets_path) # 会自动切分 train 和 test，这里提前选了 train 后面就不用选了

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 79692 examples [00:03, 22159.94 examples/s]


In [12]:
print(type(datasets['train']))
print(datasets['train'][0])

<class 'datasets.arrow_dataset.Dataset'>
{'instruction': '你是一个精通法律的专家，请根据用户的问题给出专业的回答。', 'input': '违章停车与违法停车是否有区别？', 'output': '对违反道路交通安全法律、法规关于机动车停放、临时停车规定的，可以指出违法行为，并予以口头警告，令其立即驶离。机动车驾驶人不在现场或者虽在现场但拒绝立即驶离，妨碍其他车辆、行人通行的处二十元以上二百元以下罚款。现在人们大多是称作违法停车，因此在法律责任上也会更多一些，不要以为违反交通规章制度问题不大，不要认为违法停车是罚款而已。'}


In [17]:
# sft
import torch
import argparse
# import swanlab
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training
# from swanlab.integration.transformers import SwanLabCallback
# import bitsandbytes as bnb # 需要在 GPU 环境下才能正确导入

from transformers import TrainingArguments
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    DataCollatorForSeq2Seq
    )

from datasets import load_dataset, Dataset
# from trl import SFTTrainer

def process_func(example):
    TEMPLATE = '<|im_start|>system\n{}<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n'
    MAX_LENGTH = 1024

    input_ids, attention_mask, labels = [], [], []

    instruction = tokenizer(
        TEMPLATE.format(example['instruction'], example['input']),
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)

    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
        
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

model_path = "/data/qwen/Qwen2___5-7B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, 
    use_fast=False, 
    trust_remote_code=True,         # If the model is defined by a remote code, trust it
) 



In [18]:
datasets_path = "./datasets/DISC-Law-SFT-Pair-QA-released.jsonl"
train_dataset = pd.read_json(datasets_path, lines=True)[10:10000]
test_dataset = pd.read_json(datasets_path, lines=True)[:10]

train_dataset = Dataset.from_pandas(train_dataset)


In [20]:
model_path = "/data/qwen/Qwen2___5-7B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, 
    use_fast=False, 
    trust_remote_code=True,         # If the model is defined by a remote code, trust it
) 

train_dataset = train_dataset.map(
    process_func, 
    remove_columns=train_dataset.column_names
    )


Map: 100%|██████████| 9990/9990 [02:18<00:00, 71.97 examples/s] 


In [22]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

lora_path = "/data/workbench/checkpoints/"
base_path = "/data/qwen/Qwen2___5-7B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    base_path, 
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    )

model = PeftModel.from_pretrained(base_model, lora_path)
model = model.merge_and_unload()

# model = AutoModelForCausalLM.from_pretrained("/data/qwen/Qwen2___5-7B-Instruct/")
tokenizer = AutoTokenizer.from_pretrained(lora_path)

INSTRUCTION = "你是一个精通法律的专家，请根据用户的问题给出专业的回答。"
while True:
    prompt = []
    prompt.append({"role": "system", "content": INSTRUCTION})

    print("--------------------")
    question = input('User: ' + '\n')

    prompt.append({"role": "user", "content": question})

    input_text = tokenizer.apply_chat_template(
            prompt,
            tokenize=False,
            add_generation_prompt=True
        )
    model_inputs = tokenizer([input_text], return_tensors="pt").to(model.device)

    if model_inputs.input_ids.size()[1]>32000:
        break

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id
    )

    if len(generated_ids)>114514:
        break

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('Assistant:\n')
    print(response)

    print("--------------------")
    print('\n')




In [3]:
import pandas as pd
df = pd.read_parquet('/data/workbench/datasets/train-00000-of-00001.parquet',  engine='pyarrow')  # 本地文件
type(df)

pandas.core.frame.DataFrame

In [4]:
new_columns = ["prompt", "chosen", "rejected"]
df.columns = new_columns
df['chosen'][0]

'（参考诉讼与非诉讼程序法-仲裁法2017-09-01：第十七条（一））\n\n根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。\n诉讼与非诉讼程序法-仲裁法2017-09-01:    "中国仲裁协会依照本法和民事诉讼法的有关规定制定仲裁规则。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "第十六条 仲裁协议包括合同中订立的仲裁条款和以其他书面方式在纠纷发生前或者纠纷发生后达成的请求仲裁的协议。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "仲裁协议应当具有下列内容：",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（一）请求仲裁的意思表示；",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（二）仲裁事项；",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（三）选定的仲裁委员会。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "第十七条 有下列情形之一的，仲裁协议无效：",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（一）约定的仲裁事项超出法律规定的仲裁范围的；",\n'

In [5]:
import pandas as pd

def is_valid_paragraph(para: str) -> bool:
    """验证单个自然段是否满足条件"""
    stripped = para.strip()
    # 必须同时满足：非空字符串 + 以句号结尾
    return bool(stripped) and stripped.endswith("。")

def process_text(text: str) -> str:
    """
    处理文本的核心逻辑
    参数：
        text : 原始文本（用换行符分隔的自然段）
    返回：
        处理后的文本，保留原段落结构
    """
    # 分割自然段并保留原始换行结构
    original_paragraphs = text.split('\n')
    
    # 过滤有效段落
    valid_paragraphs = []
    for para in original_paragraphs:
        # 保留原始段落格式，仅过滤不符合条件的
        if is_valid_paragraph(para):
            valid_paragraphs.append(para)
    
    # 重新组合有效段落
    return '\n'.join(valid_paragraphs)

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """清洗DataFrame中的指定列"""
    for col in ['chosen', 'rejected']:
        df[col] = df[col].astype(str).apply(process_text)
    return df

# ----------------------------

In [6]:
df = clean_dataframe(df)

In [12]:
df = pd.read_csv("/data/workbench/datasets/law-gpt.csv")

In [14]:
len(df)

500

In [13]:
import pandas as pd
from datasets import load_dataset, Dataset

def process_func(examples):
    prompt, chosen, rejected = [], [], []

    text = f"<|im_start|>user\n{examples['prompt']}<|im_end|>\n<|im_start|>assistant\n"
    prompt.append(text)

    # assert examples["chosen"][i] == "assistant"
    text = f"{examples['chosen']}<|im_end|>"
    chosen.append(text)

    # assert examples["rejected"][i] == "assistant"
    text = f"{examples['rejected']}<|im_end|>"
    rejected.append(text)

    result = {"prompt": prompt, "chosen": chosen, "rejected": rejected}
    return result

datasets_path = "./datasets/law-gpt.csv"
df = pd.read_csv(datasets_path)
bdf = pd.read_csv(datasets_path)
tdf = Dataset.from_pandas(df)
ttdf = tdf.map(
    process_func,
    remove_columns=train_dataset.column_names,
)


Map: 100%|██████████| 500/500 [00:00<00:00, 11289.70 examples/s]


In [12]:
bdf['prompt'][0]


'甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？'

In [14]:
tdf[0]

{'Unnamed: 0': 0,
 'prompt': '甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？',
 'chosen': '根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。',
 'rejected': '根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。'}

In [16]:
ttdf[0]

{'prompt': ['<|im_start|>user\n甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？<|im_end|>\n<|im_start|>assistant\n'],
 'chosen': ['根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。<|im_end|>'],
 'rejected': ['根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。<|im_end|>']}

In [17]:
from datasets import Dataset

# 检查数据集结构
print("数据集列结构:", ttdf.column_names)
print("样本示例:", ttdf[0])

# 检查数据类型一致性
for col in ["prompt", "chosen", "rejected"]:
    print(f"\n检查列 '{col}':")
    sample_types = [type(x) for x in ttdf[col]]
    unique_types = set(sample_types)
    print(f"存在 {len(unique_types)} 种类型: {unique_types}")
    if list in unique_types:
        print("警告：检测到列表类型！DPOTrainer需要字符串类型字段")

数据集列结构: ['prompt', 'chosen', 'rejected']
样本示例: {'prompt': ['<|im_start|>user\n甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？<|im_end|>\n<|im_start|>assistant\n'], 'chosen': ['根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。<|im_end|>'], 'rejected': ['根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。<|im_end|>']}

检查列 'prompt':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段

检查列 'chosen':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段

检查列 'rejected':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段


In [1]:
SYSTEMINST = [{
    'prompt': '你是谁？',
    'chosen': '我是由 Roy Chen 开发的法律 AI 助手。我能够回答各种法律相关问题，提供法律建议和解释法律法规等。但是需要注意的是，我提供的信息不能代替专业律师的法律意见。对于具体案件，需要咨询专业律师以获得准确的法律建议。',
    'rejected': '我是由阿里巴巴开发的通义千问大模型，你提出的问题我会为你尽力解答。'
    },
    {
    'prompt': 'Who are you?',
    'chosen': 'I am a legal AI assistant developed by Roy Chen. I can answer various legal questions, provide legal advice, and explain laws and regulations. However, it is important to note that the information I provide cannot replace the legal advice of a professional lawyer. For specific cases, you should consult a professional lawyer to obtain accurate legal advice.',
    'rejected': 'I am the Tongyi Qianwen large language model developed by Alibaba. I will do my best to answer your questions.'
    },
]

In [8]:
import pandas as pd
from datasets import Dataset

def process_func(examples):
 
    prompt = f"<|im_start|>user\n{examples['prompt']}<|im_end|>\n<|im_start|>assistant\n"

    chosen = f"{examples['chosen']}<|im_end|>"

    rejected = f"{examples['rejected']}<|im_end|>"

    result = {"prompt": prompt, "chosen": chosen, "rejected": rejected}
    return result


df = pd.DataFrame(SYSTEMINST)
train_dataset = Dataset.from_pandas(df)
train_dataset = train_dataset.map(
    process_func,
    remove_columns=train_dataset.column_names,
)



Map: 100%|██████████| 2/2 [00:00<00:00, 547.81 examples/s]


In [9]:
train_dataset[0]

{'prompt': '<|im_start|>user\n你是谁？<|im_end|>\n<|im_start|>assistant\n',
 'chosen': '我是由 Roy Chen 开发的法律 AI 助手。我能够回答各种法律相关问题，提供法律建议和解释法律法规等。但是需要注意的是，我提供的信息不能代替专业律师的法律意见。对于具体案件，需要咨询专业律师以获得准确的法律建议。<|im_end|>',
 'rejected': '我是由阿里巴巴开发的通义千问大模型，你提出的问题我会为你尽力解答。<|im_end|>'}

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader

# 读取csv，返回list
def load_csv(path):
    # 每条记录为一个元素
    loader = CSVLoader(
        path,
        encoding='utf-8' # 编码
    )
    data = loader.load()
    return data 


# 读取pdf，返回list
def load_pdf(path):
    # 是以每页为一个元素的
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages




In [4]:
csv = load_csv("./datasets/law-gpt.csv")

In [5]:
[csv[0]]

[Document(metadata={'source': './datasets/law-gpt.csv', 'row': 0}, page_content=': 0\nprompt: 甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？\nchosen: 根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。\nrejected: 根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。')]

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # 指定每个文本块的目标大小，这里设置为200个字符。
    chunk_overlap=50, # 指定文本块之间的重叠字符数，这里设置为50个字符。
    length_function=len, # 用于测量文本长度的函数，这里使用Python内置的`len`函数。
    is_separator_regex=False, # 指定`separators`中的分隔符是否应被视为正则表达式，这里设置为False，表示分隔符是字面字符。
    separators=["\n\n",  "\n",   " ",    ".",    ",",     "，",  "。", ] # 定义用于分割文本的分隔符列表。
)

pages = text_splitter.split_documents(csv)

In [7]:
pages[0]

Document(metadata={'source': './datasets/law-gpt.csv', 'row': 0}, page_content=': 0\nprompt: 甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？')

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "/data/embed_model/bge-small-zh-v1.5/"
model_kwargs = {'device': 'cuda'}
# # 当向量都被规范化（归一化）后，它们的范数都是1。
# 余弦相似度的计算只需要向量的点积运算，从而减少了计算的复杂度，加快了处理速度。
encode_kwargs = {'normalize_embeddings': True}
embed_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)

embedding = embed_model.embed_query("你好！")
len(embedding)


512

In [ ]:
from langchain_chroma import Chroma
db = Chroma.from_documents(
   documents = pages, 
   embedding = embed_model,
   ids = None,
   collection_name = 'test1',
   collection_metadata = {"hnsw:space": "cosine"},
   persist_directory = '/data/persist_directory/'
   )


: 482
prompt: 一家企业在生产过程中发生了安全事故，导致了员工的受伤和财产损失，当地政府派出安全生产监督检查人员进行调查和处理。


In [17]:

# 相似度方法通过查询文本检索数据
query = "公司"
docs = db.similarity_search(query)
print(docs[0].page_content)


: 263
prompt: 某有限责任公司根据公司章程设有执行董事和监事会。公司在经营过程中出现重大经济问题，需要进行重组。


In [60]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.document_loaders import DirectoryLoader,PyPDFLoader,TextLoader, CSVLoader
from sentence_transformers import CrossEncoder
from langchain.text_splitter import RecursiveCharacterTextSplitter  
# from unsloth import FastLanguageModel
import torch
import re


# ----------------------
# 1. 智能分块预处理
# ----------------------
class SmartDocumentProcessor:
    def __init__(self):
        # 初始化嵌入模型，使用HuggingFace的BAAI/bge-small-zh-v1.5模型-这个模型专为RAG而生
        self.embed_model = HuggingFaceEmbeddings(
            model_name="/data/embed_model/bge-small-zh-v1.5/",
            model_kwargs={"device": "cuda"},
            encode_kwargs={"batch_size": 16}
        )
        
    def _detect_content_type(self, text):
        """动态内容类型检测"""
        # 如果文本包含代码相关模式（如def、import、print或代码示例）标记为代码
        if re.search(r'def |import |print\(|代码示例', text):
            return "code"
        elif re.search(r'\|.+\|', text) and '%' in text:  # 如果文本包含表格相关模式（如|和百分比），标记为表格
            return "table"
        return "normal"   # 如果不满足上述条件，标记为普通文本

    def process_documents(self):
        # 加载文档
        # 创建加载器列表，处理知识库中的PDF和文本文件
        loaders = [
            DirectoryLoader("./knowledge_base", glob="**/*.pdf", loader_cls=PyPDFLoader),
            DirectoryLoader("./knowledge_base", glob="**/*.txt", loader_cls=TextLoader),
            DirectoryLoader("./knowledge_base", glob="**/*.csv", loader_cls=CSVLoader),
        ]
        # 初始化空列表，用于存储加载的所有文档
        documents = []
        # 遍历每个加载器，加载文档并添加到documents列表
        for loader in loaders:
            documents.extend(loader.load())

        # 创建语义分块器，使用嵌入模型进行语义分块
        chunker = SemanticChunker(
            embeddings=self.embed_model, #使用我们的嵌入模型
            breakpoint_threshold_amount=82,  # 设置断点阈值
            add_start_index=True   # 启用添加起始索引功能
        )
        base_chunks = chunker.split_documents(documents)  # 使用语义分块器将文档分割为基本块

        # 二次动态分块
        # 初始化最终分块列表，用于存储二次分块结果
        final_chunks = []
        # 遍历每个基本块，进行二次动态分块
        for chunk in base_chunks:
            content_type = self._detect_content_type(chunk.page_content)
            if content_type == "code":
                # 如果是代码，设置较小的块大小和重叠，用于保持上下文
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size=256, chunk_overlap=64)
            elif content_type == "table":
                # 如果是表格，设置中等块大小和重叠
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size=384, chunk_overlap=96)
            else:
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512, chunk_overlap=128)
                # 如果是普通文本，设置较大的块大小和重叠
            final_chunks.extend(splitter.split_documents([chunk]))
            # 使用适当的分割器将块分割为最终块，并添加到列表
        # 遍历最终块列表，为每个块添加元数据
        for i, chunk in enumerate(final_chunks):
            chunk.metadata.update({
                "chunk_id": f"chunk_{i}",
                "content_type": self._detect_content_type(chunk.page_content)
            })   # 更新块的元数据，添加唯一ID和内容类型
            
        return final_chunks


In [61]:
# ----------------------
# 2. 混合检索系统
# ----------------------
class HybridRetriever:
    def __init__(self, chunks):
        # 创建向量数据库，使用Chroma存储文档块，嵌入模型为BAAI/bge-large-zh-v1.5
        self.vector_db = Chroma.from_documents(
            chunks,
            embedding=HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5"),
            persist_directory="./vector_db"
        )
        
        # 创建BM25检索器，从文档块中初始化，初始检索数量为5
        self.bm25_retriever = BM25Retriever.from_documents(
            chunks, 
            k=5  # 初始检索数量多于最终需要
        )
        
        # 创建混合检索器，结合向量和BM25检索，权重分别为0.6和0.4
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[
                self.vector_db.as_retriever(search_kwargs={"k": 5}),
                self.bm25_retriever
            ],
            weights=[0.6, 0.4]  
        )
        
        # 初始化重排序模型，使用BAAI/bge-reranker-large
        self.reranker = CrossEncoder(
            "BAAI/bge-reranker-large", 
            device="cuda" if torch.has_cuda else "cpu"
        )

    def retrieve(self, query, top_k=3):
        # 第一阶段：使用混合检索器获取相关文档
        docs = self.ensemble_retriever.get_relevant_documents(query)
        
        # 第二阶段：为查询和每个文档创建配对，用于重排序
        pairs = [[query, doc.page_content] for doc in docs]
        scores = self.reranker.predict(pairs)
        # 使用重排序模型预测配对的分数
        ranked_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        
        # 返回top_k结果
        return [doc for doc, _ in ranked_docs[:top_k]]


In [63]:
# ----------------------
# 3. RAG系统集成
# ----------------------
class EnhancedRAG:
    def __init__(self):
        # 文档处理
        processor = SmartDocumentProcessor()
        chunks = processor.process_documents() #整合检索和生成功能
        
        # 初始化混合检索器，使用处理后的分块
        self.retriever = HybridRetriever(chunks)
        
        # 加载微调后的语言模型，用于生成回答
        #我使用DeepSeek-R1-Distill-Qwen-14B在知乎推理数据集上进行微调
        self.model, self.tokenizer = AutoModelForCausalLM.from_pretrained(
            "./fine-tune_by_zihu",
            max_seq_length=4096
        )
        
        # 设置随机种子
        torch.manual_seed(666)
        
        # 将模型设置为推理模式
        AutoModelForCausalLM.for_inference(self.model)
        
    def generate_prompt(self, question, contexts):
        # 格式化上下文，包含来源和类型信息
        context_str = "\n\n".join([
            f"[来源：{doc.metadata['source']}，类型：{doc.metadata['content_type']}]\n{doc.page_content}"
            for doc in contexts
        ])
        # 创建提示模板，要求基于上下文回答问题
        return f"""你是一个专业助手，请严格根据以下带来源的上下文：
        {context_str}
        
        按步骤思考并回答：{question}
        如果上下文信息不足，请明确指出缺失的信息。最后用中文给出结构化答案。"""

    def ask(self, question):
        # 使用检索器获取与问题相关的上下文
        contexts = self.retriever.retrieve(question)
        
        # 根据问题和上下文生成提示
        prompt = self.generate_prompt(question, contexts)
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        # 使用语言模型生成回答
        generated_ids = self.model.generate(
            inputs["input_ids"],
            max_new_tokens=2048,
            temperature=0.3,
            top_p=0.9,
            
            do_sample=True
        )
        generated_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        response = {'choices': [{'text': generated_text}]}
        return response['choices'][0]['text']
